# Visión por Computador - Práctica 1
## Filtrado y muestreo

### A) Implementar una función de convolución que debe ser capaz de calcular la convolución 2D de una imagen con una máscara. Ahora supondremos que la máscara es extraída del muestreo con una Gaussiana 2D simétrica. Para ello implementaremos las siguientes funciones auxiliares:

#### 1. Cálculo del vector máscara: Sea $f(x) = e^{\left(-0.5\frac{x^2}{\sigma^2}\right)}$ una funcion donde $\sigma$ representa un parámetro en unidades píxel. Implementar una función que tomando $\sigma$ como parámetro de entrada devuelva una máscara de convolución representativa de dicha función. Justificar los pasos dados.

#### 2. Implementar una función que calcule la convolución de un vector señal 1D con un vector-máscara 1D de longitud inferior al de la señal usando dos posibles tipos de condiciones de entorno (uniforme a ceros y reflejada). La salida será un vector de igual longitud que el vector señal de entrada. Pasos:

   ##### a) Implementar una función que calcule la convolución 1D entre dos vectores, dando de salida solo los valores donde ha sido posible el cálculo.
    
   ##### b) Definir un vector auxilar de longitud mayor definida a partir de las dimensiones de los dos vectores de entrada, p.e. la señal tiene longitud $N$ y el otro $2k + 1$ entonces el vector auxiliar será $N+2k$. Copiar en su centro el vector señal y rellenar los extremos usando el criterio de borde elegido. 
    
   ##### c) Ejecutar la función del paso a sobre el vector creado y la máscara. En el caso de que el vector de entrada sea de color, habrán de extraerse cada uno de los tres vectores correspondientes a cada uno de ellos y volver a montar el vector de salida. Usar las funciones *split()* y *merge()* de openCV.
    
#### 3. Implementar una función que tomando como entrada una imagen y el valor de $\sigma$ calcule la convolución de dicha imagen con uma máscara Gaussiana 2D. Usar las funciones implementadas en el punto anterior. Recordar que la Gaussiana es descomponible en convoluciones 1D por filas y columnas.

- **Apartado 1**: Para resolver el apartado 1, se ha creado la función ***get_mask_vector*** que recibe como parámetro de entrada el valor de $\sigma$. Una vez dentro de la función, muestrearemos $f(x)$ entre los valores $-3\sigma$ y $3\sigma$.

    Para ello, en el intervalo $\left[\lfloor-3\sigma\rfloor, \lceil3\sigma\rceil\right]$, muestrearemos los $n$ valores enteros que haya en este intervalo, obteniendo un vector de enteros. Sobre este vector de enteros, aplicaremos la función $f(x)$ a este vector obteniendo así la máscara, pero sin normalizar.

    Una vez obtenida, los valores que se encuentren por debajo de $|3\sigma|$, los llevaremos a 0, ya que consideraremos que son poco relevantes y que influyen muy poco en el resultado. Tras esto, normalizaremos a 1 la máscara y se devolverá.

In [1]:
# Devuelve el vector gaussiano de la máscara
def get_mask_vector(sigma):
    # Obtenemos el valor "límite" que puede tener un
    # píxel para que sea significativo
    limit_3sigma = fx(3 * sigma, sigma)
    # Obtenemos un array de valores discretos
    # para realizar la gaussiana
    aux = np.arange(math.floor(-3 * sigma), math.ceil(3 * sigma )+ 1)
    # Rellenamos la máscara aplicando la exponencial
    mask = np.array([fx(i, sigma) for i in aux])
    # Despreciamos los valores menores a 3 sigma
    mask[mask < limit_3sigma] = 0
    # Normalizamos la máscara a 1
    mask = np.divide(mask, np.sum(mask))
    # Devolvemos la máscara
    return mask

- **Apartado 2**: para resolver el apartado dos, primero tenemos que fijarnos en cómo se realiza una convolución. $$G[i,j] = \sum_{u=-k}^k\sum_{v=-k}^kH[u,v]F[i-u,j-v]$$